# Report - Bloom Filter

    
   ### Bloom Filter
   **Bloom Filter** is a probabilistic data structure that is used to see if certain information is in a data set, which means that it can tell us with certainty that the element is not in the set but not so much if it is.
    
   How does a Bloom Filter work? It combines ideas from HashSets and bitsets into a single data structure. So, we create a bitset with m bits in it, and then we’ll hash each element using k hashing functions. Now each element occupies k bitset entries. 
    
   When we want to check if an element is present in the filter, we must use the same hashing functions used in the entire set. If all entries determine 1, the element is in the bloom filters. But if one or more entries determine a 0, the element isn’t present in the filter. 
    
   In some cases, all entries may determine 1 with an element that isn’t in the filter. This is when the probabilistic nature of the filter manifests itself, by giving us false positives. This probability can be calculated using the variables of numbers of hash functions, k, capacity, c, and the number of bits, m:  $$ {(1\ -\ e^{-kc/m})}^k $$
    
   Because it’s very space-efficient since the elements themselves are not added to a set but rather a hash of the elements, they are good for applications where space is needed, and the false positive probability is not a concern.

   ### Possible application of the algorithm:

   The implemented application emulates a software whose goal is to prevent frauds in the transactions that go through a banking institution. This detection is made through a verication of a person's NIF.

   A simple implementation of this would be to put every known NIF classified as fraudulent in a database and for every transaction, we analyze to see if the NIF involved in the transaction is in the set. Being aware of the false positive probability, the bank will then verify if the transactions classified as fraudulent are so indeed.
   
   There have been provided two small and simple interfaces. One to mock a user's use case and another to mock a bank employee's use case.
   
   In this mock implementation the filter has been loaded with **5 million** entries. Even with this ammount of entries, the loading time for NIF lookups is extremely short, being almost instantaneous, thus demonstrating the power of the algorithm to deal with large ammounts of data.

   **User's use case:**
   
   When someone is trying to make a transaction, the banking software makes a call to an "API" whose goal is to verify the validity of the inserted NIF. In this implementation, the call will be made directly to the bloom filter's method **is_member()**, but in the real world this should be done with an higher level of abstraction, thus using an API.
    
   If the tax identification number (NIF) of the person that is performing the transaction exists in a set of NIF’s classified as fraudulent, then that transaction should be denied, giving the user an option to appeal to the banking institution. Otherwise, the transaction can be approved, with certainty that the inserted NIF is not fraudulent. This certainty is ensured by the nature of bloom filters, which cannot give false positives.
   
   Another feature at the user's disposal is the reporting option. If they have been scammed by a certain NIF, they can report it to the banking institution. After a certain number of reports, that NIF is automatically added to the filter.
   
   **Employee's use case:**
    
   The bank employee has three features at his disposal. The first one is to see a list of appeals that have been made since it last logged on to the system. The second is to check reported NIFs that have not yet achieved the threshold, having the possibility to confirm or whitelist them. The last option at his disposal is to check some statistics regarding the filter. 
    
    
   ### Packages used:
    
   **Math** is a Python package that provides access to mathematical functions. This package has been used in order to implement the mathematical functions that define the optimal number of bits ***m*** and the optimal number of hash functions ***k***, in order to store ***n*** different entries with ***p*** probability of false positives.
    
   **Mmh3** is a Python library for MurmurHash with a set of fast and robust hash functions. These hash functions are based on two simple operations, multiplication and rotation. Due to their simplicity, these hash function are more suited to be used in algorithms such as bloom filters, rather than being used in criptographic applications. 
   
   This package has been used in order to hash the NIFs prior to being added to bloom filter. This hashing of the NIFs is what determines which bits are to be set to one on the bloom filter.
    
   **Bitarray** is a library that provides an object type which efficiently represents an array of Booleans. This package as been used in order to create an array of bits, all set to zero, in order to implement the bloom filter algorithm.

    


# Algorithm Implementation

### Import Section

In [ ]:
# Section dedicated to the imports
import mmh3
import math
from bitarray import bitarray

### Bloom Filter implementation

In [ ]:

class BloomFilter:
    
    # Initializes the BloomFilter
    def __init__(self, num_itens, error_percent):
        # Bloom filter size for N members with X % probability of error, according to the formula present on Wikipedia
        self.size = math.ceil(-(num_itens * math.log(error_percent/100))/(math.log(2)**2))
        
        # Calculate the ideal number of hash function, according to the formula present on Wikipedia
        self.num_hashes = round((self.size/num_itens) * math.log(2))
        
        # Instance a bitarray with the calculated size a set it all to zeros
        self.filter = bitarray(self.size)
        self.filter.setall(0)
        self.num_members = 0
        
    
    
    # Hashes the NIB and adds it to the filter.
    def add_element(self, nib):
        for t in range(self.num_hashes):
            index = mmh3.hash(nib, t) % self.size
            self.filter[index] = 1
        self.num_members += 1
        return
    
    
    # Check whether the NIB is present in the filter
    def is_member(self, nib):
        for t in range(self.num_hashes):
            index = mmh3.hash(nib, t) % self.size
            if self.filter[index] == 0:
                return False
        return True
    
    # Auxiliary functions just to see some statistics regarding the filter.
    def get_bloom(self):
        return self.filter
    
    def get_num_hashes(self):
        return self.num_hashes
    
    def get_filter_size(self):
        return self.size
    
    def get_number_of_members(self):
        return self.num_members
    

### Some small tests

In [ ]:
# Tests just to see if the base implementation is working as expected.

# Instancing the bloom filter for 3 elements and 1% chance of false positives
bloom = BloomFilter(3, 1)

# Adding some mock NIBs to the filter
nib = 'PT 42531'
nib2 = 'PT 3455'
nib3 = 'PT 23452'
bloom.add_element(nib)
bloom.add_element(nib)
bloom.add_element(nib2)

# Printing some statistics
print('Contents of the filter:\n->',bloom.get_bloom())
print('Filter size:\n->', bloom.get_filter_size())
print('Ideal number of hashes:\n->', bloom.get_num_hashes())

# Checking if the elements are present or not
print(f'\nIs \"{nib}\" a member of the filter?\n->',bloom.is_member(nib))
print(f'\nIs \"{nib2}\" a member of the filter?\n->',bloom.is_member(nib2))
print(f'\nIs \"{nib3}\" a member of the filter?\n->',bloom.is_member(nib3))


### Instancing the filter for our application

In [ ]:
# Initial instancing
size = 5001000
prob = 0.001

bloom_filter = BloomFilter(size, prob)

# Inital data import
with open('data_nifs.csv', 'r') as file:
    for line in file:
        bloom_filter.add_element(str(line.strip()))


In [ ]:
# Auxiliar structures for the application
appeal_nifs = []
reported_nifs = {}

In [ ]:
# list of fraudulent nifs extracted from the csv
#[728822881, 994241283, 333328302, 991793366, 789461803, 758735022, 684045672, 724369198, 775089419, 295959770]

### User's Interface

In [ ]:
print('Welcome to the TMBD bank of choice!\nWe are now running a service to check whether a NIF is fraudulent.')
while True:
    choice = input('\nDo you: \n1) Intend to make a transaction \n2) Report a fraudulent NIF\n3) Quit\n\n')
    if choice == '1':
        print('Please enter your receipt data.\n')
        name = input('Name: ')
        nif = input('NIF: ')
        ammount = input('Ammount: ')
        
        print('\nPlease hold while we verify your transaction...\n')
        if bloom_filter.is_member(nif):
            print('The inserted NIF has been identified as a fraudulent one.')            
            answer = input('Do you wish to appeal? [Y/N] \n')
            if answer == 'Y':
                appeal_nifs.append(nif)
                print('Your appeal has been registered.\n')
        else:
            print('Your transaction has been approved!\n')
            
    elif choice == '2':
        fraud_nif = input('\nPlease enter the NIF you want to report:\n')
        
        if fraud_nif not in reported_nifs.keys():
            reported_nifs[fraud_nif] = 1
        else:
            if (reported_nifs[fraud_nif] + 1) >= 5:
                bloom_filter.add_element(fraud_nif)
                reported_nifs.pop(fraud_nif)
            else:
                reported_nifs[fraud_nif] += 1

        print('\nThank you for your report!')

    elif choice == '3': 
        break

### Employee's Interface

In [ ]:
print('Hello employee.')
while True:
    op = input('\nWhat do you want to check? \n1) Appealed NIFs \n2) Reported NIFs \n3) Filter statistics \n4) Quit\n\n')
    if op == '1':
        print(appeal_nifs)
    elif op == '2':
        print(reported_nifs)
        if reported_nifs != {}:
            ans = input('Do you wish to [R]emove or [C]onfirm a NIF?\n')
        
            if ans.lower() == 'c':
                nif = input('Please enter the NIF you want to mark as fraudulent: \n')
                bloom_filter.add_element(nif)
                print(f'The following NIF has been added to the filter: {nif}')
        
            elif ans.lower() == 'r':
                nif = input('Please enter the NIF you want to whitelist: \n')
                reported_nifs.pop(nif)
                print(f'The following NIF has been whitelisted: {nif}')

    elif op == '3':
        print(f'\nThe filter is using {bloom_filter.get_filter_size()} bits')
        print(f'\nWe are currently using {bloom_filter.get_num_hashes()} hash functions.')
        print(f'\nWe have identified {bloom_filter.get_number_of_members()} fraudulent NIFs.')
    elif op == '4':
        break